In [1]:
import pandas as pd
from datetime import datetime
data = pd.read_csv('./HW2 附件/HW2_hr-analytics_train.csv')

In [14]:
df = data.copy()
# check nan
def check_nan(df):
    return df.isna().any().any()

def onehot_encoding(df, column_name):
    df_encoded = pd.get_dummies(df, columns=[column_name], prefix=[column_name])
    print(df.shape, "->", df_encoded.shape)
    return df_encoded

def pprint(output = '\n', show_time = False): # print and fprint at the same time
    global filename
    print(output)
    with open(filename, 'a') as f:
        if show_time:
            f.write(datetime.now().strftime("[%Y-%m-%d %H:%M:%S] "))

        f.write(str(output))
        f.write('\n')
        
def check_non_numeric_values(df, column_name):
    """
    Check if any value in the specified column is not int or float.

    Parameters:
    df (DataFrame): The DataFrame to check.
    column_name (str): The name of the column to check.

    Returns:
    bool: True if any non-numeric value is found, False otherwise.
    """
    # Get the specified column
    column = df[column_name]

    # Check if any value is not int or float
    non_numeric_values = column[~column.apply(lambda x: isinstance(x, (int, float)))]
    
    return not non_numeric_values.empty

col_not_num = []
for col in df.columns:
    if check_non_numeric_values(df, col):
        col_not_num.append(col)
        
print(f"Doing onehot encoding on col: {col_not_num}")
for col in col_not_num:
    df = onehot_encoding(df, col)


Doing onehot encoding on col: ['sales', 'salary']
(10000, 10) -> (10000, 19)
(10000, 19) -> (10000, 21)
